In [ ]:
import requests
from bs4 import BeautifulSoup

url = "http://divui.com/blog/72-dia-diem-du-lich-sai-gon-moi-ve-dem-mien-phi-tong-hop-tu-a-z/"
rq = requests.get(url)
doc = BeautifulSoup(rq.text)

doc = doc.find('div', {"class":"td-post-content"})

Lưu miêu tả của từng địa điểm vào một danh sách

In [ ]:
p_tag = doc.find_all('p')[3:-5]
description = []

for item in p_tag:
  if (item.find('a') == None or "Cần Giờ là" in item.get_text()) and item.get("class") == None:
    description.append(item.get_text())

Lưu link ảnh của từng địa điểm vào một danh sách

In [ ]:
figure_tag = doc.find_all('figure')
img_src  = []

for item in figure_tag:
  img_src.append(item.find('img').get("data-src"))

Lưu tên của từng địa điểm vào một danh sách

In [ ]:
h3_tag = doc.find_all('h3')
place = []

for item in h3_tag:
  place.append(item.get_text())

Lưu phân loại của từng địa điểm vào một danh sách

In [ ]:
h2_tag = doc.find_all('h2')[1:]
type_of_place = []

for item in h2_tag:
  type_of_place.append(item.get_text())

Lưu giá vé và địa chỉ của từng địa điểm vào hai danh sách riêng biệt. Địa điểm thiếu thông tin thì sẽ được lưu bằng:

*   "No Ticket Needed" với thiếu giá vé
*   "No Infomation From Website" với thiếu địa chỉ



In [ ]:
ul_tag = doc.find_all('ul')[:-2]

li_fee = []
li_address = []

for item in ul_tag:
  if (item.find('li', {"style":"list-style-type: none;"})):
    continue
  li_tag = item.find_all('li')
  if (len(li_tag) == 2):
    li_fee.append(li_tag[0].get_text().split("vé:")[1][1:])
    li_address.append(li_tag[1].get_text().split(":")[1][1:])
  elif (len(li_tag) == 1 and "Giá vé" in li_tag[0].get_text()):
    li_fee.append(li_tag[0].get_text().split(": ")[1])
  else:
    li_address.append(li_tag[0].get_text().split(":")[1][1:])

index_without_address = [31, 37, 41, 42]
address = []
index = -1
for i in range(72):
  if i in index_without_address:
    address.append("No Information From Website")
  else:
    index = index + 1
    address.append(li_address[index])

index_with_fee = [0, 1, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                  18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 34, 35, 36]
ticket_price = []
index = -1
for i in range(72):
  if i in index_with_fee:
    index = index + 1
    ticket_price.append(li_fee[index])
  else:
    ticket_price.append("No Ticket Needed")

Đưa dữ liệu vào bảng

In [ ]:
data = []
for i in range(72):
  data_itr = dict()
  data_itr['Place'] = place[i]
  data_itr['Address'] = address[i]

  if (i <= 14):
    data_itr['Type'] = type_of_place[0]
  elif (i <= 31):
    data_itr['Type'] = type_of_place[1]
  elif (i <= 42):
    data_itr['Type'] = type_of_place[2]
  else:
    data_itr['Type'] = type_of_place[3]
    
  data_itr['Ticket Price'] = ticket_price[i]
  data_itr['Image Source'] = img_src[i]
  data_itr['Description'] = description[i]
  


  data.append(data_itr)

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
#df.to_csv("CrawlData_divui.csv", encoding="utf-8-sig", index = False)
df

,Place,Address,Type,Ticket Price,Image Source,Description
0,Bitexco Skydeck,"2 Hải Triều, Bến Nghé, Hồ Chí Minh",Địa điểm du lịch Sài Gòn mới,200.000/ người,http://divui.com/blog/wp-content/uploads/2016/...,Sài Gòn Skydeck cho bạn cơ hội thu vào tầm mắt...
1,Bảo tàng tranh 3D Art in us,"02-04, Đường số 9, KDC Himlam, Phường Tân Hưng...",Địa điểm du lịch Sài Gòn mới,200.000/ người,http://divui.com/blog/wp-content/uploads/2018/...,Art in us sẽ là không gian để bạn bung tỏa hết...
2,World Of Heineken,"Bitexco Financial Tower, 2 Hải Triều, Bến Nghé...",Địa điểm du lịch Sài Gòn mới,No Ticket Needed,http://divui.com/blog/wp-content/uploads/2018/...,Khám phá lịch sử phát triển hãng Heineken – hã...
3,Vietopia,"khu dân cư Him Lam, quận 7",Địa điểm du lịch Sài Gòn mới,"ngày thường 190.000/ trẻ em, cuối tuần 280.000...",http://divui.com/blog/wp-content/uploads/2018/...,"Là mô hình công viên vui chơi kiểu mới, tạo mô..."
4,KizCiti,"KCV Khánh Hội, 48 Hoàng Diệu, Phường 5, Quận 4...",Địa điểm du lịch Sài Gòn mới,"thứ 2 đến thứ 6: 150.000/ trẻ em; thứ 7, chủ n...",http://divui.com/blog/wp-content/uploads/2018/...,KizCiti – thành phố trẻ em được xây dựng như l...
...,...,...,...,...,...,...
67,Chợ Dân Sinh,"104 Yersin, phường Nguyễn Thái Bình, quận 1",Địa điểm du lịch Sài Gòn miễn phí,No Ticket Needed,http://divui.com/blog/wp-content/uploads/2018/...,"Nằm ngay trung tâm Sài Gòn, chợ Dân Sinh bán đ..."
68,Chợ Nhật Tảo,"Đường Nguyễn Tri Phương, Phường 4, Quận 10, Th...",Địa điểm du lịch Sài Gòn miễn phí,No Ticket Needed,http://divui.com/blog/wp-content/uploads/2018/...,Chuyên đồ điện tử và bất kỳ thứ gì có vi mạch ...
69,Chợ Bình Điền,"Khu Phố 6, Nguyễn Văn Linh, P. 7, Quận 8, TP. HCM",Địa điểm du lịch Sài Gòn miễn phí,No Ticket Needed,http://divui.com/blog/wp-content/uploads/2018/...,Chợ đầu mối Bình Điền chỉ hợp lúc sáng sớm với...
70,Chợ hoa Hồ Thị Kỷ,"Hồ Thị Kỷ, Phường 1, Quận 10, Tp. HCM",Địa điểm du lịch Sài Gòn miễn phí,No Ticket Needed,http://divui.com/blog/wp-content/uploads/2018/...,"Với hàng trăm kiot hoa với đủ loại hoa, lá tra..."
